<a href="https://colab.research.google.com/github/Liza-IITP/Dynamic_Pricing_Parking_Lots/blob/main/MODEL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------ INSTALLATION ------------------
!pip install ydata-profiling pathway bokeh panel --quiet

# ------------------ IMPORTS ------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

# ------------------ LOAD DATA ------------------
df = pd.read_csv('data.csv')

# ------------------ FEATURE ENGINEERING ------------------
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

# Handle missing VehicleType categories
vehicle_types = ["bike", "car", "cycle", "truck"]
vehicle_dummies = pd.get_dummies(df["VehicleType"], prefix="VehicleType").astype(int)
for vt in vehicle_types:
    col = f"VehicleType_{vt}"
    if col not in vehicle_dummies:
        vehicle_dummies[col] = 0
df = pd.concat([df, vehicle_dummies], axis=1)

traffic_map = {'low': 0, 'average': 1, 'high': 2}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

df["DayOfWeek"] = df["Timestamp"].dt.dayofweek

df.drop(columns=["Unnamed: 0", "Latitude", "Longitude", "VehicleType",
                 "LastUpdatedDate", "LastUpdatedTime", "TrafficConditionNearby"],
        inplace=True)

df.drop_duplicates(inplace=True)
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Save to CSV for Pathway replay
df.to_csv("parking_stream_all_spots.csv", index=False)


In [ ]:

# ------------------ ALPHA ESTIMATION ------------------
df["SimulatedPrice"] = np.nan
spot_alpha_list = []

for spot_id in df["SystemCodeNumber"].unique():
    sub_df = df[df["SystemCodeNumber"] == spot_id].copy()
    base_price = 10
    alpha_dummy = 1.0
    price_series = [base_price]
    for occ in sub_df["OccupancyRate"]:
        price_series.append(price_series[-1] + alpha_dummy * occ)
    sub_df["SimulatedPrice"] = price_series[1:]
    sub_df["DeltaPrice"] = sub_df["SimulatedPrice"].diff()
    sub_df["OccupancyRate_lag"] = sub_df["OccupancyRate"].shift(1)
    sub_df = sub_df.dropna()

    x = sub_df["OccupancyRate_lag"].values
    y = sub_df["DeltaPrice"].values
    alpha = np.sum((x - x.mean()) * (y - y.mean())) / np.sum((x - x.mean())**2)
    spot_alpha_list.append(alpha)

alpha_avg = np.mean(spot_alpha_list)
print(f"📈 Average Estimated Alpha for All Spots: {alpha_avg:.4f}")

In [ ]:

# ------------------ PATHWAY STREAM SETUP ------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRate: float
    Capacity: int
    QueueLength: int
    IsSpecialDay: int
    TrafficLevel: int
    DayOfWeek: int
    VehicleType_bike: int
    VehicleType_car: int
    VehicleType_cycle: int
    VehicleType_truck: int

data = pw.demo.replay_csv("parking_stream_all_spots.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t=data.Timestamp.dt.strptime(fmt),
    day=data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
# ------------------ MODEL-1: DAILY PRICE BY SPOT ------------------
model1_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.SystemCodeNumber, pw.this.day),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber=pw.reducers.any(pw.this.SystemCodeNumber),
        total_occ=pw.reducers.sum(pw.this.OccupancyRate),
    )
    .with_columns(
        Price_Model1 = 10 + alpha_avg * pw.this.total_occ
    )
)

In [ ]:

# ------------------ VISUALIZATION ------------------
def plot_model1(source):
    fig = bokeh.plotting.figure(
        height=500,
        width=900,
        title="Model-1: Dynamic Price for All 14 Parking Spots (Daily)",
        x_axis_type="datetime"
    )
    for code in source.data["SystemCodeNumber"]:
        fig.scatter("t", "Price_Model1", source=source, size=5, color="navy", legend_label="Dynamic Price")
    fig.xaxis.axis_label = "Date"
    fig.yaxis.axis_label = "Price"
    return fig

viz_model1 = model1_window.plot(plot_model1, sorting_col="t")
pn.Column(viz_model1).servable()

In [ ]:
%%capture --no-display
pw.run()